# Bayesian estimation algorithm for the phase and the visibilities in a q-plates system

This code is meant to be run on a GPU.

In [3]:
import torch
import numpy as np
import cvxpy as cp
from scipy.stats import bootstrap
from torch.distributions.multivariate_normal import MultivariateNormal
import math
import matplotlib.pyplot as plt
from IPython.display import clear_output

gpu = torch.device("cuda:0") 

In [4]:
def estimate_mean(batchsize, weights, particles, nparameters):
    """
    Mean of the positions of the particles
    weights.shape -> (batchsize, nparticles)
    particles.shape -> (batchsize, nparticles, nparameters)
    return (batchsize, nparameters)
    """
    #we distinguish the theta parameter from the others
    #* is the element-wise multiplication
    mean_theta = torch.remainder(torch.angle(torch.sum((torch.exp(1.0j*particles[:, :, 0:1])*weights[:, :, None]), dim=1)), 2*math.pi) #(shape -> batchsize)
    #the above exponentials in the weighted mean are necessary because theta is a circular variable
    #weighted mean values of the visibilities
    mean_visibilities = torch.sum(particles[:, :, 1:nparameters]*weights[:, :, None], dim=1, dtype=torch.double) #shape -> batchsize*nparameters-1
    #returns the concatenated arrays
    return torch.cat((mean_theta, mean_visibilities), dim=1)

In [5]:
def estimate_covariance(batchsize, weights, particles, nparticles, nparameters):
    """
    Computes the covariant matrix
    weights.shape -> (batchsize, nparticles)
    particles.shape -> (batchsize, nparticles, nparameters)
    """
    #broadcasted version on the mean to all the particles in a batch
    broadcasted_mean = torch.broadcast_to(estimate_mean(batchsize, weights, particles, nparameters)[:, None, :], (batchsize, nparticles, nparameters))
    #broadcasted weights to all the parameters
    broadcasted_weights = torch.broadcast_to(weights[:, :, None], (batchsize, nparticles, nparameters))
    #the difference between the particle value and the angula mean has to be computed with the circular distance
    diff_theta = math.pi-torch.abs(torch.remainder(particles[:, :, 0:1] - broadcasted_mean[:, :, 0:1], 2*math.pi)-math.pi)
    #difference of the particles visibilities and their mean
    diff_visibilities = particles[:, :, 1:nparameters] - broadcasted_mean[:, :, 1:nparameters]
    diff = torch.cat((diff_theta, diff_visibilities), dim=2)
    #this matrix product return the estimated covariance matrix
    return torch.transpose(broadcasted_weights*diff, 1, 2)@diff         

In [6]:
def prob_outcomes(batchsize, outcomes, particles, qplates, phase, qresources, nparticles, gpu):
    """
    Probability of measurement
    outcomes.shape -> (batchsize, 1)
    particles.shape -> (batchsize, nparticles, nparameters)
    qplates.shape -> (batchsize, 1)
    phase.shape -> (batchsize, 1)
    qresources.shape -> (nparameters-1, )
    the number of q-plates available is the number of parameter we are estimating - 1 (the phase)
    return a tensor of size (batchsize, nparticles), containing the probabilities of outcome for each of the particles
    """
    #define broadcasted versions of outcomes, measurement, and qplates
    outcomes_broad = torch.broadcast_to(outcomes, (batchsize, nparticles))
    phase_broad = torch.broadcast_to(phase, (batchsize, nparticles))
    qplates_broad = torch.broadcast_to(qplates, (batchsize, nparticles))
    #the qplate numer is used to select the correct visibility
    visibilities = torch.take_along_dim(particles, qplates_broad.long()[:, :, None], dim = 2)[:, :, 0]
    #here we define the tensors for the result and for the phase
    out_set = torch.tensor([-1.0, +1.0], dtype=torch.double, device=gpu)
    phase_set = torch.tensor([0.0, math.pi/2], dtype=torch.double, device=gpu)
    #return the tensor contain the probability of having observed outcomes with the given measurements for each particle.
    return (1.0+out_set[outcomes_broad.long()]*visibilities*torch.cos(particles[:, :, 0]*qresources[(qplates_broad-1).long()]+phase_set[phase_broad.long()]))/2.0

In [7]:
def bayesian_update(batchsize, weights, outcomes, particles, qplates, phase, qresources, nparticles):
    """
    Returns the updated weights, applies the Bayesian rule
    """
    tmp_weights = prob_outcomes(batchsize, outcomes, particles, qplates, phase, qresources, nparticles, gpu)*weights
    new_weights = tmp_weights/torch.sum(tmp_weights, dim=1)[:, None]
    return new_weights

In [8]:
def resample_simplified(batchsize, weights, particles, nparticles, nparameters, a, gpu):
    """
    Simplified resampling
    it produce as output
    newParticles.shape -> (batchsize, nparticles, nparameters)
    """
    #sampled particles
    pos = torch.multinomial(weights, nparticles, replacement=True)
    #we need to broadcast the selection along the dimention of the parameters, this is necessary in order to apply the follwoing function
    pos_broadcasted = torch.broadcast_to(pos[:, :, None], (batchsize, nparticles, nparameters))
    #this filters the tensor particles, in the second direction according to the dimension 1
    new_particles = torch.take_along_dim(particles, pos_broadcasted, dim=1)
    new_weights = torch.tensor(np.full((batchsize, nparticles), 1/nparticles), dtype=torch.double, device=gpu)
    return new_particles, new_weights

In [9]:
def resample(batchsize, weights, particles, nparticles, nparameters, a, gpu):
    """
    Circular resampling algorithm with concentration towards the mean
    """
    mean = estimate_mean(batchsize, weights, particles, nparameters)
    h = math.sqrt(1-a**2)
    reduced_cov = (h**2)*estimate_covariance(batchsize, weights, particles, nparticles, nparameters)
    #sampled particles
    pos = torch.multinomial(weights, nparticles, replacement=True)
    #we need to broadcast the selection along the dimention of the parameters, this is necessary in order to apply the follwoing function
    pos_broadcasted = torch.broadcast_to(pos[:, :, None], (batchsize, nparticles, nparameters))
    #this filters the tensor particles, in the second direction according to the dimension 1
    new_particles = torch.take_along_dim(particles, pos_broadcasted, dim=1)
    #this new particles should be now mixed with a broadcasted version of the mean
    broadcasted_mean = torch.broadcast_to(estimate_mean(batchsize, weights, particles, nparameters)[:, None, :], (batchsize, nparticles, nparameters))
    mean_gaussian = torch.zeros(batchsize, nparticles, nparameters, dtype=torch.double, device=gpu)
    #linear combination of the visibilitie
    mean_gaussian[:, :, 1:nparameters] = a*new_particles[:, :, 1:nparameters] + (1-a)*broadcasted_mean[:, :, 1:nparameters]
    #linear combinations of the angles
    mean_gaussian[:, :, 0] = torch.remainder(torch.angle(a*torch.exp(1.0j*new_particles[:, :, 0]) + (1-a)*torch.exp(1.0j*broadcasted_mean[:, :, 0])), 2*math.pi)
    #now we sample from a multivariate Gaussian with mean contained in meanGaussian, and covariance matrix the brodcasted version of reducedCov
    reduced_cov_broad = torch.broadcast_to(reduced_cov[:, None, :, :], (batchsize, nparticles, nparameters, nparameters))
    #new particles
    normal = MultivariateNormal(mean_gaussian, reduced_cov_broad)
    #the angles have to be casted in [0, 2*pi] and the visibilities in [0, 1]
    new_particles = normal.sample()
    new_particles[:, :, 0] = torch.remainder(new_particles[:, :, 0], 2*math.pi)
    #how can we broadcast the operations min and max? We can use a mask
    pos_over_1 = (new_particles[:, :, 1:nparameters] > 1.0).long()
    new_particles[:, :, 1:nparameters] = new_particles[:, :, 1:nparameters]*(1-pos_over_1)+pos_over_1
    pos_under_0 = (new_particles[:, :, 1:nparameters] < 0.0).long()
    new_particles[:, :, 1:nparameters] = new_particles[:, :, 1:nparameters]*(1-pos_under_0)
    new_weights = torch.tensor(np.full((batchsize, nparticles), 1/nparticles), dtype=torch.double, device=gpu)
    return new_particles, new_weights

In [10]:
def utility(batchsize, weights, particles, qplates, phase, Q, nparticles, nparameters, qresources, gpu):
    """
    Function that compute the utility function of the list of measurements defined by the tensors qplates and phase
    weights.shape -> (batchsize, nparticles)
    particles.shape -> (batchsize, nparticles, nparameters)
    qplates.shape -> (batchsize, 1)
    phase.shape -> (batchsize, 1)
    Q.shape -> (nparameters, nparameters)
    """
    u_outcomes = torch.zeros(batchsize, 2, dtype=torch.double, device=gpu)
    for out in range(2):
        #broadcasted version of the output
        out_broad = torch.broadcast_to(torch.tensor([out], device=gpu), size=(batchsize, 1))
        #broadcasted version of the weight matrix
        Q_broad = torch.broadcast_to(Q[None, :, :], size=(batchsize, nparameters, nparameters))
        hyp_weights = bayesian_update(batchsize, weights, out_broad, particles, qplates, phase, qresources, nparticles)
        hyp_covariance = estimate_covariance(batchsize, hyp_weights, particles, nparticles, nparameters)
        hyp_nvariance = -((torch.diagonal(hyp_covariance@Q_broad, dim1=1, dim2=2)).sum(dim=1))
        prob_out = prob_outcomes(batchsize, out_broad, particles, qplates, phase, qresources, nparticles, gpu)
        u_outcomes[:, out] = hyp_nvariance*torch.diagonal(weights@(torch.transpose(prob_out, 0, 1)), dim1=0, dim2=1)
    return torch.sum(u_outcomes, dim=1)

In [27]:
def optimize(batchsize, weights, particles, nparticles, nparameters, Q, qresources, gpu):
    """
    Function that choses the optimal measurements, according to the utility function
    """
    #in this tensor we save the utility for each of the possible measurements, and each of the estimation in the batch
    utility_tensor = torch.zeros(batchsize, 2*qresources.shape[0], dtype=torch.double, device=gpu)
    #qp = 0, 1, 2, 3
    for qp in range(qresources.shape[0]):
        #r = 0, 1
        for p in range(2):
            qplates = torch.tensor(np.full((batchsize, 1), qp+1), device=gpu)
            phase = torch.tensor(np.full((batchsize, 1), p), device=gpu)
            utility_tensor[:, 2*qp+p] = utility(batchsize, weights, particles, qplates, phase, Q, nparticles, nparameters, qresources, gpu)
    #take the argmax of utilityTensor in dim=1
    optimal_meas = torch.argmax(utility_tensor, axis=1)
    phase = torch.remainder(optimal_meas, 2)
    qplates = (optimal_meas-phase)/2+1
    return qplates[:, None], phase[:, None]

In [28]:
def simulation(batchsize, qplates, phase, trueangles, visibilities, qresources, gpu):
    """
    Simulation of the experiment. 
    """
    #computation of the probability of the 0 outcome, corrisponding to -1
    #here we define the tensors for the result and for the phase
    out_set = torch.tensor([-1.0, +1.0], dtype=torch.double, device=gpu)
    phase_set = torch.tensor([0.0, math.pi/2], dtype=torch.double, device=gpu)
    #probability of having observed outcomes with the given measurements for each particle.
    prob_out_0 = (1.0-1.0*visibilities[(qplates-1).long()]*torch.cos(trueangles*qresources[(qplates-1).long()]+phase_set[phase.long()]))/2.0
    p = torch.rand(batchsize, 1, device=gpu)
    #true if outcome 0 happens
    outcomes = p<prob_out_0
    outcomes = 1 - outcomes.int()
    return outcomes

In [29]:
def run_estimation(batchsize, nparticles, nparameters, a, resample_threshold, Q, trueangles, visibilities, qresources, nsteps, gpu):
    """
    This function runs an instance of the estimation procedure
    """
    #the weight are uniform at the beginning
    weights = torch.tensor(np.full((batchsize, nparticles), 1/nparticles), dtype=torch.double, device=gpu)
    #the particles are initialized at random
    particles = torch.rand(batchsize, nparticles, nparameters, dtype=torch.double, device=gpu)
    particles[:, :, 0] = 2*math.pi*particles[:, :, 0]
    #initial measurement
    qplates = torch.tensor(np.full((batchsize, 1), 1), device=gpu)
    phase = torch.tensor(np.full((batchsize, 1), 0), device=gpu)
    #number of photon used in the estimation
    for i in range(nsteps):
        #simulation of the esperiment
        outcomes = simulation(batchsize, qplates, phase, trueangles, visibilities, qresources, gpu)
        #Bayesian update of the weights of the particle filter
        weights = bayesian_update(batchsize, weights, outcomes, particles, qplates, phase, qresources, nparticles)
        #computation of the next optimal measurement in a "greedy" fashion
        qplates, phase = optimize(batchsize, weights, particles, nparticles, nparameters, Q, qresources, gpu)
        #checking the condition for resampling
        #true means we need to resample the simulation, which corresponds to 1, and therefore gets counter among the non-null values
        array_resample = (1/torch.sum(weights**2, dim=1) < resample_threshold*nparticles).long()
        #to_resample -> (batch_resample, ) -> position of the experiments to be resampled
        to_resample = torch.nonzero(array_resample)[:, 0]
        #number of simulations to resamples
        batch_resample = to_resample.size(0)
        #if there are any
        if (batch_resample > 0):
            #broadcasting of the index to filter the weights
            to_resample_broad_1 = torch.broadcast_to(to_resample[:, None], (batch_resample, nparticles))
            #broadcasting to filter the particles
            to_resample_broad_2 = torch.broadcast_to(to_resample[:, None, None], (batch_resample, nparticles, nparameters))
            #experiments to resample
            resample_weights = torch.take_along_dim(weights, to_resample_broad_1, dim=0)
            resample_particles = torch.take_along_dim(particles, to_resample_broad_2, dim=0)
            #resampling
            resample_particles, resample_weights = resample_simplified(batch_resample, resample_weights, resample_particles, nparticles, nparameters, a, gpu)
            #inserting the new values back in particles and weights
            linear_index = torch.arange(0, batch_resample, 1, device=gpu)
            particles[to_resample.long(), :, :] = resample_particles[linear_index.long(), :, :]
            weights[to_resample.long(), :] = resample_weights[linear_index.long(), :]
        #progressing of the estimation
        #print((np.round(i/nsteps*100)).astype(int), "%")
        #clear_output(wait=True)
   
    return estimate_mean(batchsize, weights, particles, nparameters)

In [30]:
def run_estimation_resources(batchsize, nparticles, nparameters, a, resample_threshold, Q, trueangles, visibilities, qresources, max_resources, gpu):
    """
    This function runs an instance of the estimation procedure and keeps track of the resources used
    """
    #the weight are uniform at the beginning
    weights = torch.tensor(np.full((batchsize, nparticles), 1/nparticles), dtype=torch.torch.double, device=gpu)
    #the particles are initialized at random
    particles = torch.rand(batchsize, nparticles, nparameters, dtype=torch.double, device=gpu)
    particles[:, :, 0] = 2*math.pi*particles[:, :, 0]
    #initial measurement
    qplates = torch.tensor(np.full((batchsize, 1), 1), device=gpu)
    phase = torch.tensor(np.full((batchsize, 1), 0), device=gpu)
    #the following array is of shape (batchsize, max_resources, nparameters) and is meant to contain the results of the estimations
    #we keep track of the current number of resources used in total for each back and use it as indeces
    results_resources = torch.zeros(batchsize, max_resources, nparameters, dtype=torch.double, device=gpu)
    #total number of resources used
    current_resources = torch.tensor(np.full(batchsize, 0), dtype=torch.long, device=gpu)
    selected_resources = torch.zeros(batchsize, max_resources, 4, dtype=torch.double, device=gpu)
    #the looping is stopped only when the first batch reaches max_resources
    #with this condition we stop too early we ask for the median to reach max_resources, but then we need to discard the batches that have gone too far.
    print("Estimating...")
    #while (torch.median(current_resources) < max_resources):
    #while(((current_resources > max_resources).long()).sum(dim=0) < 0.99*batchsize):
    while(torch.quantile(current_resources.float(), 0.01) < max_resources):
        #simulation of the esperiment
        outcomes = simulation(batchsize, qplates, phase, trueangles, visibilities, qresources, gpu)
        #Bayesian update of the weights of the particle filter
        weights = bayesian_update(batchsize, weights, outcomes, particles, qplates, phase, qresources, nparticles)
        #computation of the next optimal measurement in a "greedy" fashion
        qplates, phase = optimize(batchsize, weights, particles, nparticles, nparameters, Q, qresources, gpu)
        #updating the number of resources
        current_resources += qresources[(qplates-1).long()[:, 0]]
        #saving the partial result at the current value of the total number of resources
        #in this line we go out of index with the while condition we chose
        #results_resources[torch.arange(0, batchsize, 1), current_resources.long() - torch.remainder(current_resources.long(), delta_n)+deltan/2, :]
        #we select the batches that have't gone too far
        #with this masking trick the results of the simulations that exceed max_resources are stored in position resources = 0, and are eliminated at the end
        mask = (current_resources < max_resources).long()
        masked_resources = mask*current_resources
        results_resources[torch.arange(0, batchsize, 1), masked_resources.long(), :] = estimate_mean(batchsize, weights, particles, nparameters)
        selected_resources[torch.arange(0, batchsize, 1), masked_resources.long(), (qplates-1).long()[:, 0]] += 1
        #checking the condition for resampling
        #true means we need to resample the simulation, which corresponds to 1, and therefore gets counter among the non-null values
        array_resample = (1/torch.sum(weights**2, dim=1) < resample_threshold*nparticles).long()
        #to_resample -> (batch_resample, ) -> position of the experiments to be resampled
        to_resample = torch.nonzero(array_resample)[:, 0]
        #number of simulations to resamples
        batch_resample = to_resample.size(0)
        #if there are any
        if (batch_resample > 0):
            #broadcasting of the index to filter the weights
            to_resample_broad_1 = torch.broadcast_to(to_resample[:, None], (batch_resample, nparticles))
            #broadcasting to filter the particles
            to_resample_broad_2 = torch.broadcast_to(to_resample[:, None, None], (batch_resample, nparticles, nparameters))
            #experiments to resample
            resample_weights = torch.take_along_dim(weights, to_resample_broad_1, dim=0)
            resample_particles = torch.take_along_dim(particles, to_resample_broad_2, dim=0)
            #resampling
            resample_particles, resample_weights = resample(batch_resample, resample_weights, resample_particles, nparticles, nparameters, a, gpu)
            #inserting the new values back in particles and weights
            linear_index = torch.arange(0, batch_resample, 1, device=gpu)
            particles[to_resample.long(), :, :] = resample_particles[linear_index.long(), :, :]
            weights[to_resample.long(), :] = resample_weights[linear_index.long(), :]
        #progressing of the estimation
        #print(torch.round(100*((current_resources > max_resources).long()).sum(dim=0)/(0.9*batchsize)), "%")
        #torch.quantile(current_resources, 0.05) > max_resources
        #print("1% quantile of the used resources:", int(torch.quantile(current_resources.float(), 0.01).item()))
        #clear_output(wait=True)
    print("Done.")
    results_resources[:, 0, :] = 0
    return results_resources, current_resources, selected_resources

In [31]:
def visualization_filter(batch, particles, weights, nparticles, nparameters):
    """
    Visualization of the phase distribution represented by the particle filter
    """
    lattice_step = 100
    hist_values = torch.zeros(lattice_step).cpu()
    particles_cpu = particles.cpu()
    weights_cpu = weights.cpu()
    for i in range(nparticles):
        hist_values[torch.floor(lattice_step*(particles_cpu[batch, i, 0]/(2*math.pi))).long()] += weights_cpu[batch, i]
    plt.plot(torch.arange(0, 2*math.pi, 2*math.pi/lattice_step).cpu(), hist_values)

In [33]:
def filter_error(scalar_error, non_zero_mask, new_batchsize):
    """
    This function filters away the resource number that do not correspond to any experiment and resamples the result
    """
    #here a threshold conditions could be inserted
    non_null_resources = torch.nonzero((non_zero_mask.sum(dim=0) > 0).long())[:, 0]
    #filtering of the resources and of the mask
    filtered_error = torch.take_along_dim(scalar_error, non_null_resources[None, :], dim=1)
    filtered_mask = torch.take_along_dim(non_zero_mask, non_null_resources[None, :], dim=1)
    #resampling of the exprimental results
    resampled_positions = torch.transpose(torch.multinomial(torch.transpose(filtered_mask, 0, 1).double(), new_batchsize, replacement=True), 0, 1)
    full_results = torch.take_along_dim(filtered_error, resampled_positions, dim=0)
    return full_results

In line diff_results[:, :, 0, 0] = diff_results[:, :, 0, 0]/2 we divided by two the error on the phase because we are actually interested in the error on $\theta$, since we are passing $2\theta$ as true phase. The errors on the visibilities should NOT be similarly halfed.

In [34]:
def estimation(batchsize, nparticles, nparameters, a, resample_threshold, Q, Qout, phase, visibilities, qresources, max_resources, cluster_size, cluster_cut, new_batchsize, gpu):
    """
    This function perform the simulation and produces a polished version of the outcome, that is the error as a function of the number of employed resources 
    """
    #trueangles, index = torch.sort(2*math.pi*torch.rand(batchsize, 1, dtype=torch.double, device=gpu), dim=0)
    #batch of true angles, all equal to phase
    trueangles = phase*torch.ones(batchsize, 1, dtype=torch.double, device=gpu)
    #mean = run_estimation(batchsize, nparticles, nparameters, a,resample_threshold, Q, trueangles, qresources, 200, gpu)
    results_resources, current_resources, _ = run_estimation_resources(batchsize, nparticles, nparameters, a, resample_threshold, Q, trueangles, visibilities, 
                                                                    qresources, max_resources, gpu)
    #it is sufficient to check the phase to be zero (the estimated visibilities will also be zero)
    #we assume of course that exactly 0 is not a return value of the algorithm   
    non_zero_mask = (1-(results_resources == 0).long())[:, :, 0]
    #computation of the scalar error
    visibilities = torch.tensor([0.900, 0.875, 0.850, 0.765], dtype=torch.double, device=gpu)
    vis_broad = torch.broadcast_to(visibilities, (batchsize, nparameters-1))
    true_param = torch.cat((trueangles, vis_broad), dim=1)
    diff_results = results_resources[:, :, :, None] - true_param[:, None, :, None]
    #the difference in the theta parameter must be corrected to account for the circular nature of the parameter
    diff_results[:, :, 0, 0] = math.pi-torch.abs(torch.remainder(diff_results[:, :, 0, 0], 2*math.pi)-math.pi)
    #Achtung: this line mist be added to account for the fact that we are estimation 2*theta but we are interested in the error on theta
    diff_results[:, :, 0, 0] = diff_results[:, :, 0, 0]/2
    #this scalar error is meaningful only for the non zero values, indeed we filter it accordingly
    scalar_error = (torch.transpose(diff_results, 2, 3)@Qout[None, None, :, :]@diff_results)[:, :, 0, 0]
    #filtering of the scalar error
    scalar_error = non_zero_mask*scalar_error
    #the scalar error needs to be clustered
    #we group the values in (batch, 30000) in groups of 20 each
    #for the data analysis to work cluster_cut should be a multiple of cluster_size, and max_resources should also be a multiple of cluster_size
    num_step = int(max_resources/cluster_size)
    scalar_error_clustered = torch.zeros(batchsize*cluster_size, num_step, device=gpu)
    non_zero_mask_clustered = torch.zeros(batchsize*cluster_size, num_step, device=gpu)
    new_resources = torch.arange(cluster_size/2, max_resources, cluster_size)
    for j in range(num_step):
        scalar_error_clustered[:, j] = torch.reshape(scalar_error[:, (j*cluster_size):((j+1)*cluster_size)], (batchsize*cluster_size, ))
        non_zero_mask_clustered[:, j] = torch.reshape(non_zero_mask[:, (j*cluster_size):((j+1)*cluster_size)], (batchsize*cluster_size, ))
    effective_batchsize = int(torch.min(torch.sum(non_zero_mask_clustered, axis=0)).item())
    #print(torch.sum(non_zero_mask_clustered, axis=0))
    clustered_error_1 = filter_error(scalar_error[:, 0:cluster_cut], non_zero_mask[:, 0:cluster_cut], new_batchsize)
    clustered_error_2 = filter_error(scalar_error_clustered, non_zero_mask_clustered, new_batchsize)
    #An important sanity check is that the size of clustered_error_1 should be cluster_cut-1, and that of clustered_error_2 max_resources/cluster_size
    if(not (clustered_error_1.shape[1] == (cluster_cut-1) and (clustered_error_2.shape[1] == int(max_resources/cluster_size)))):
        return
    cluster_error = torch.cat((clustered_error_1, clustered_error_2[:, int(cluster_cut/cluster_size):]), axis=1)
    resources = torch.cat((torch.arange(1, cluster_cut, 1), new_resources[int(cluster_cut/cluster_size):]))
    number_points = cluster_error.shape[1]
    resources_bar = torch.zeros(number_points, dtype=torch.double, device=gpu)
    resources_bar[cluster_cut:] = cluster_size/2
    return resources.cpu().numpy(), resources_bar.cpu().numpy(), cluster_error.cpu().numpy(), effective_batchsize

It is important that the new_batchsize is not greater than the minimum number of actual measurements in a cluster of data. Otherwise we are oversamping the data and therefore the confidence interval won't be accurate. The median is chosen to limitate the importance of the outliers in the estimation.

In [35]:
def control_room(batchsize, new_batchsize, Q, Qout, max_resources):
    """
    All the true parameters of the estimation, the quantum resources, 
    the visibilities and the technical parameters that make the estimation work are hidden in this function.
    Here we also compute the bayesian bound for the media square error.
    Q is the weights function used for the estimation, Qout is used for the visualization of the results.
    Usually Q=Qout
    """
    phases = 2*np.array([0.00235, 0.06145, 0.29345, 0.21635, 0.38000, 0.49620, 0.46280, 1.13975, 
                      1.26430, 1.44890, 1.66450, 1.75520, 1.87500, 2.12710, 2.58995, 2.74000, 2.96000])
    num_phases = phases.size
    #up to batchsize = 10000 should be supported by the gpu
    #batchsize = 1000
    #nparticles = 5000
    nparticles = 5000
    #the number of parameters should be one more than the 
    visibilities = torch.tensor([0.900, 0.875, 0.850, 0.765], dtype=torch.double, device=gpu)
    qresources = torch.tensor([1, 2, 11, 51], dtype=torch.long, device=gpu)
    nparameters = 5
    a = 0.9
    resample_threshold = 0.5
    cluster_size = 50
    cluster_cut = 100
    #the new batchsize should be comparable with the size of the smaller clustered batch, this can be verified a posteriori
    phases_error = np.zeros((new_batchsize, (cluster_cut-1)+int(max_resources/cluster_size)-int(cluster_cut/cluster_size)))
    for i in range(num_phases): 
        #sum the error for diferent angles, in order to get one sample of delta_1 + delta_2 + ... delta_J
        #both the resource_bar and the resources should be the same for every estimation
        resources, resources_bar, cluster_error, effective_batchsize = estimation(batchsize, nparticles, nparameters, a, resample_threshold, Q, Qout, phases[i], visibilities, 
                                                                                  qresources, max_resources, cluster_size, cluster_cut, new_batchsize, gpu)
        print("Phase", i+1, "/", num_phases, "completed.")
        print("Size of the smaller effective batch:", effective_batchsize)
        phases_error += cluster_error/num_phases
    coeff_I_theta = np.zeros(4)
    coeff_I_vis = np.zeros(4)
    #this line accounts for the halfed phase
    for i in range(4):
        coeff_I_theta[i] = 2*qresources.cpu().numpy()[i]**2*(1-np.sqrt(1-(visibilities.cpu().numpy()[i])**2));
        coeff_I_vis[i] = 2*(1-np.sqrt(1-(visibilities.cpu().numpy()[i])**2))/((visibilities.cpu().numpy()[i])**2*np.sqrt(1-(visibilities.cpu().numpy()[i])**2));
    nu = cp.Variable(4)
    van_trees_expression = (Q.cpu().numpy()[0, 0]*cp.inv_pos(cp.sum(cp.multiply(nu, coeff_I_theta)))/4+
                            Q.cpu().numpy()[1, 1]*cp.inv_pos(cp.multiply(nu, coeff_I_vis)[0])+
                            Q.cpu().numpy()[2, 2]*cp.inv_pos(cp.multiply(nu, coeff_I_vis)[1])+
                            Q.cpu().numpy()[3, 3]*cp.inv_pos(cp.multiply(nu, coeff_I_vis)[2])+
                            Q.cpu().numpy()[4, 4]*cp.inv_pos(cp.multiply(nu, coeff_I_vis)[3]))
    objective = cp.Minimize(van_trees_expression)
    constraints = [nu >= 0, 2*cp.sum(cp.multiply(qresources.cpu().numpy(), nu)) == 1]
    prob = cp.Problem(objective, constraints)
    #The optimal objective value is returned by `prob.solve()`.
    van_trees_bound = 0.454896*prob.solve(verbose=False)
    #now we implement the bootstrap procedure
    res = bootstrap((phases_error, ), np.median, axis=0, confidence_level=0.99, n_resamples=1000, method='basic')
    #median error to be plotted, this is chosen to reduce the fluactuations.
    median = np.median(phases_error, axis=0)
    #lower confidence interval
    ci_l = res.confidence_interval[0]
    #higher confidence interval
    ci_h = res.confidence_interval[1]
    #plot
    plt.xlabel("Resources")
    plt.ylabel("MSE")
    plt.xscale('log')
    plt.yscale('log')
    plt.plot(resources[10:], median[10:], 'b-')
    plt.fill_between(resources[10:], ci_l[10:], ci_h[10:], color='r', alpha=.2)
    
    #we plot the SQL and the HS just as comparison.
    #plt.plot(resources[10:], 1/resources[10:], 'g')
    #plt.plot(resources[10:], 1/resources[10:]**2, 'r')
    #plt.plot(resources[10:], van_trees_bound/resources[10:], 'k')
    file_name = ("estimation" + str(int(Q.cpu().numpy()[0, 0]))+
                 str(int(Q.cpu().numpy()[1, 1]))+
                 str(int(Q.cpu().numpy()[2, 2]))+
                 str(int(Q.cpu().numpy()[3, 3]))+
                 str(int(Q.cpu().numpy()[4, 4]))+ ".svg")
    plt.savefig(file_name)
    return resources, median, ci_l, ci_h, van_trees_bound

In [36]:
nparameters = 5

In [ ]:
#Estimation of the phase and the first visibility, but only the visibility is plot
Q = torch.zeros(nparameters, nparameters, dtype=torch.double, device=gpu)
Q[0, 0] = 1
Q[1, 1] = 1
resources, median, ci_l, ci_h, van_trees_bound = control_room(30, 100, Q, Q, 30000)

In [ ]:
#Estimation of the phase and the second visibility, but only the visibility is plot
Q = torch.zeros(nparameters, nparameters, dtype=torch.double, device=gpu)
Q[0, 0] = 1
Q[2, 2] = 1
resources, median, ci_l, ci_h, van_trees_bound = control_room(40, 100, Q, Q, 30000)

In [ ]:
#Estimation of the phase and the third visibility, but only the visibility is plot
Q = torch.zeros(nparameters, nparameters, dtype=torch.double, device=gpu)
Q[0, 0] = 1
Q[3, 3] = 1
resources, median, ci_l, ci_h, van_trees_bound = control_room(70, 100, Q, Q, 30000)

In [22]:
#Estimation of the phase and the fourth visibility but only the visibility is plot
Q = torch.zeros(nparameters, nparameters, dtype=torch.double, device=gpu)
Q[0, 0] = 1
Q[4, 4] = 1
resources, median, ci_l, ci_h, van_trees_bound = control_room(100, 100, Q, Q, 30000)

In [ ]:
#estimation of all the parameters, but only the visibilities are plot
Q = torch.ones(nparameters, nparameters, dtype=torch.double, device=gpu)
control_room(100, 100, Q, Q, 30000)

Estimating...


## Plot of the q-plate use as a function of the resource number

In [ ]:
#This data analysis is meant to produce the array selected_resources, from which we build some interesting plots.
batchsize = 1000
nparticles = 5000
nparameters = 5
a = 0.9
resample_threshold = 0.5
Q = torch.zeros(nparameters, nparameters, dtype=torch.double, device=gpu)
Q[0, 0] = 1
phase_number = 1
visibilities = torch.tensor([0.900, 0.875, 0.850, 0.765], dtype=torch.double, device=gpu)
qresources = torch.tensor([1, 2, 11, 51], dtype=torch.long, device=gpu)
max_resources = 30000
simulation_bool = False

selected_resources_sum = torch.zeros(batchsize, max_resources, 4, dtype=torch.double, device=gpu)

phases_list = np.array([2.35000e-03, 6.14500e-02, 3.80000e-01, 4.96200e-01, 1.66450e+00, 1.87500e+00, 2.58995e+00, 2.96000e+00])
for i in range(8):
    trueangles = phases_list[i]*torch.ones(batchsize, 1, dtype=torch.double, device=gpu)
    results_resources, _, selected_resources = run_estimation_resources(batchsize, nparticles, nparameters, a, resample_threshold, Q, trueangles, visibilities, qresources, max_resources, gpu)
    selected_resources_sum += selected_resources

summed = torch.sum(selected_resources_sum, dim=0)
#the y-axis is the number of uses of a certain q-plate as a function of the total number of resources
plt.plot(summed[1:, 0].cpu().numpy())
plt.plot(summed[1:, 1].cpu().numpy())
plt.plot(summed[1:, 2].cpu().numpy())
plt.plot(summed[1:, 3].cpu().numpy())
plt.xscale('log')
plt.yscale('log')
plt.savefig("frequency10000.svg")